<h3>Geocoding Function</h3>

Based on web scraping technique and google map api

Given an address, return the corresponding formatted address and latitude, longitude

In [38]:
def get_geolocation_data(address_string, format, country='all', types=False): 
    address_string='_'.join(address_string.split(' '))
    url="https://maps.googleapis.com/maps/api/geocode/%s?address=%s" % (format.lower(),address_string)
    import requests
    response=requests.get(url)
    if response.status_code==200:
        
        #when format=='json', restore the wanted data in x_list
        if format.lower()=='json':  
            json_data=response.json()
            x_list=[]
            if types==True:
                for x in json_data['results']:
                    types_list=[y for y in x['types']]
                    x_list.insert(len(x_list),(x['formatted_address'],x['geometry']['location']['lat'],x['geometry']['location']['lng'],types_list))   
            else:
                x_list=[(x['formatted_address'],x['geometry']['location']['lat'],x['geometry']['location']['lng']) for x in json_data['results']]

        #when format=='xml', restore the wanted data in x_list
        elif format.lower()=='xml':
            xml_data=response.content     
            from lxml import etree
            root=etree.XML(xml_data)   
            x_list=[]
            for y in root.iter('result'):
                if types==True:
                    types_list = [item.text for item in y.findall('type')]
                    temporary_list=(y.find('formatted_address').text,y.find('geometry/location/lat').text,y.find('geometry/location/lng').text,types_list)
                else:
                    temporary_list=(y.find('formatted_address').text,y.find('geometry/location/lat').text,y.find('geometry/location/lng').text)
                x_list.insert(len(x_list),(temporary_list))
        
        #according to the input values of country, restore the ultimate outpue in add_list
        add_list=[]
        if country.lower()=='all':
                add_list=x_list
        else:
                for y in x_list:
                    if country in y[0].split()[-1]:
                        add_list.insert(len(add_list),y)
        return add_list
    
    else:
        return None


<h5>Let's try to search "Columbia"</h5>

In [54]:
address_string='Columbia'
get_geolocation_data(address_string, format='json', country='all', types=False)

[('Columbia, MO, USA', 38.9517053, -92.3340724),
 ('Columbia, SC, USA', 34.0007104, -81.0348144),
 ('Columbia, MD, USA', 39.2037144, -76.86104619999999),
 ('Colombia', 4.570868, -74.297333),
 ('Columbia, TN 38401, USA', 35.6150716, -87.0352831)]

<h5>If we only want to know the "Columbia" in United States, then set country='USA'</h5>

In [60]:
address_string='Columbia University'
get_geolocation_data(address_string, format='json', country='all', types=True)

[('116th St & Broadway, New York, NY 10027, USA',
  40.8075355,
  -73.9625727,
  ['establishment', 'point_of_interest', 'university'])]